### Implementing RAG using multiple documents and choramdb

In [1]:
import os

## Open AI KEY
os.environ['OPENAI_API_KEY'] = "YOUR_API_KEY"

In [2]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader, DirectoryLoader

Loading the Documents

In [3]:
loader = DirectoryLoader("new_articles/",glob = './*.txt',loader_cls=TextLoader)
documnets = loader.load()

In [4]:
## printing the first document
print(documnets[0])
print('Total Documnets-->',len(documnets))

page_content='Signaling that investments in the supply chain sector remain robust, Pando, a startup developing fulfillment management technologies, today announced that it raised $30 million in a Series B round, bringing its total raised to $45 million.\n\nIron Pillar and Uncorrelated Ventures led the round, with participation from existing investors Nexus Venture Partners, Chiratae Ventures and Next47. CEO and founder Nitin Jayakrishnan says that the new capital will be put toward expanding Pando’s global sales, marketing and delivery capabilities.\n\n“We will not expand into new industries or adjacent product areas,” he told TechCrunch in an email interview. “Great talent is the foundation of the business — we will continue to augment our teams at all levels of the organization. Pando is also open to exploring strategic partnerships and acquisitions with this round of funding.”\n\nPando was co-launched by Jayakrishnan and Abhijeet Manohar, who previously worked together at iDelivery,

Splitting the text into chunks


In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000 , chunk_overlap = 200)
texts = text_splitter.split_documents(documnets)

In [6]:
print("lenght of documnet after splitting-->",len(texts))

lenght of documnet after splitting--> 233


creating DB

In [7]:
# embed and store the texts
# supplying a presist_dir will store the embeddings on disk

presist_dir = 'db' ## Folder name would be db

## using OpenAI embeddings
embeddings = OpenAIEmbeddings()



c:\Users\M_ros\anaconda3\envs\AssignmentLangChan\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
vectordb = Chroma.from_documents(documents = texts,
                                 embedding = embeddings,
                                 persist_directory = presist_dir)
## it will store all the documents we have coded 

In [9]:
# persist the db to disk

vectordb.persist()
vectordb = None

In [10]:
# now we can load the 
vectordb = Chroma(persist_directory=presist_dir,embedding_function=embeddings)

## Making a retriver

In [11]:
retriver = vectordb.as_retriever()

In [12]:
docs = retriver.get_relevant_documents("How much moeny did Pando raise")

In [13]:
docs

[Document(page_content='Signaling that investments in the supply chain sector remain robust, Pando, a startup developing fulfillment management technologies, today announced that it raised $30 million in a Series B round, bringing its total raised to $45 million.\n\nIron Pillar and Uncorrelated Ventures led the round, with participation from existing investors Nexus Venture Partners, Chiratae Ventures and Next47. CEO and founder Nitin Jayakrishnan says that the new capital will be put toward expanding Pando’s global sales, marketing and delivery capabilities.\n\n“We will not expand into new industries or adjacent product areas,” he told TechCrunch in an email interview. “Great talent is the foundation of the business — we will continue to augment our teams at all levels of the organization. Pando is also open to exploring strategic partnerships and acquisitions with this round of funding.”', metadata={'source': 'new_articles\\05-03-ai-powered-supply-chain-startup-pando-lands-30m-invest

In [14]:
len(docs)

4

In [15]:
retriver = vectordb.as_retriever(search_kwargs = {'k':2})

In [16]:
retriver.search_type

'similarity'

In [17]:
retriver.search_kwargs

{'k': 2}

## Making a Chain

In [18]:
# created a chain to answer question
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                       chain_type = 'stuff',
                                       retriever = retriver,
                                       return_source_documents = True)

c:\Users\M_ros\anaconda3\envs\AssignmentLangChan\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [32]:
## Cite sources
def process_llm_responce(llm_responce):
    print(llm_responce['result'])
    print('\n\nSources:')
    for s in llm_responce['source_documents']:
        print(s.metadata['source'])

## Example

In [33]:
query = 'What is hugging face'
llm_re = qa_chain(query)
process_llm_responce(llm_re)

 Hugging Face is a large platform for hosting and running AI models.


Sources:
new_articles\05-03-spawning-lays-out-its-plans-for-letting-creators-opt-out-of-generative-ai-training.txt
new_articles\05-04-hugging-face-and-servicenow-release-a-free-code-generating-model.txt
